In [3]:
import cv2
import numpy as np
import os
from skimage import exposure
from skimage.filters import laplace
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd

class PIVVideoAnalyzer:
    """
    Clase para analizar videos y evaluar su idoneidad para análisis PIV
    mediante el cálculo de métricas de calidad.
    """
    
    def __init__(self, videos_dir):
        """
        Inicializa el analizador con el directorio de videos.
        
        Args:
            videos_dir (str): Ruta al directorio que contiene los videos a analizar
        """
        self.videos_dir = videos_dir
        self.results = {}
    
    def analyze_videos(self, roi_mask=None):
        """
        Analiza todos los videos del directorio y calcula métricas de calidad.
        
        Args:
            roi_mask (numpy.ndarray, optional): Máscara binaria que delimita la región de interés.
                                              1 para la piscina, 0 para el fondo.
        
        Returns:
            dict: Diccionario con los resultados por video
        """
        video_files = [f for f in os.listdir(self.videos_dir) 
                      if f.endswith(('.mp4', '.avi', '.mov'))]
        
        for video_file in video_files:
            print(f"Analizando video: {video_file}")
            video_path = os.path.join(self.videos_dir, video_file)
            self.results[video_file] = self.analyze_single_video(video_path, roi_mask)
            
        return self.results
    
    def analyze_single_video(self, video_path, roi_mask=None):
        """
        Analiza un único video y calcula las métricas de calidad.
        
        Args:
            video_path (str): Ruta al archivo de video
            roi_mask (numpy.ndarray, optional): Máscara binaria de la región de interés
        
        Returns:
            dict: Diccionario con las métricas calculadas
        """
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"Error: No se pudo abrir el video {video_path}")
            return {}
        
        # Obtener información básica del video
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        
        # Inicializar resultados
        metrics = {
            'global_contrast_std': [],
            'global_contrast_range': [],
            'local_contrast': [],
            'illumination_uniformity': [],
            'sharpness': [],
            'background_temporal_variance': []
        }
        
        # Si no se proporciona máscara, crear una máscara por defecto (todo el frame es ROI)
        if roi_mask is None:
            roi_mask = np.ones((height, width), dtype=np.uint8)
        elif roi_mask.shape[:2] != (height, width):
            print("Error: Las dimensiones de la máscara no coinciden con las del video")
            roi_mask = cv2.resize(roi_mask, (width, height))
        
        # Matriz para almacenar valores temporales para cálculo de varianza de fondo
        background_pixels = np.logical_not(roi_mask).astype(np.bool_)
        if np.any(background_pixels):
            temporal_values = []
        
        # Procesar cada frame
        frame_count = 0
        while True:
            ret, frame = cap.read()
            if not ret or frame_count >= total_frames:
                break
            
            # Convertir a escala de grises si es necesario
            if len(frame.shape) == 3:
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            else:
                gray = frame
            
            # 1. Contraste Global
            std_dev = np.std(gray)
            value_range = np.max(gray) - np.min(gray)
            metrics['global_contrast_std'].append(std_dev)
            metrics['global_contrast_range'].append(value_range)
            
            # 2. Contraste Local usando CLAHE
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
            clahe_img = clahe.apply(gray)
            local_contrast = np.std(clahe_img)
            metrics['local_contrast'].append(local_contrast)
            
            # 3. Uniformidad de Iluminación
            grid_size = 16  # Dividir en una cuadrícula de 16x16
            grid_h, grid_w = height // grid_size, width // grid_size
            region_means = []
            
            for i in range(grid_size):
                for j in range(grid_size):
                    region = gray[i*grid_h:(i+1)*grid_h, j*grid_w:(j+1)*grid_w]
                    region_means.append(np.mean(region))
            
            illumination_uniformity = np.std(region_means)
            metrics['illumination_uniformity'].append(illumination_uniformity)
            
            # 4. Nitidez usando Laplaciano
            laplacian = cv2.Laplacian(gray, cv2.CV_64F)
            sharpness = np.var(laplacian)
            metrics['sharpness'].append(sharpness)
            
            # 5. Varianza temporal en regiones fuera de la piscina
            if np.any(background_pixels):
                background_values = gray[background_pixels]
                temporal_values.append(np.mean(background_values))
            
            frame_count += 1
        
        # Calcular varianza temporal si hay píxeles de fondo
        if np.any(background_pixels) and temporal_values:
            metrics['background_temporal_variance'] = np.var(temporal_values)
        else:
            metrics['background_temporal_variance'] = 0
        
        # Calcular estadísticas agregadas para las métricas por frame
        results = {
            'global_contrast_std_mean': np.mean(metrics['global_contrast_std']),
            'global_contrast_std_var': np.var(metrics['global_contrast_std']),
            'global_contrast_range_mean': np.mean(metrics['global_contrast_range']),
            'global_contrast_range_var': np.var(metrics['global_contrast_range']),
            'local_contrast_mean': np.mean(metrics['local_contrast']),
            'local_contrast_var': np.var(metrics['local_contrast']),
            'illumination_uniformity_mean': np.mean(metrics['illumination_uniformity']),
            'illumination_uniformity_var': np.var(metrics['illumination_uniformity']),
            'sharpness_mean': np.mean(metrics['sharpness']),
            'sharpness_var': np.var(metrics['sharpness']),
            'background_temporal_variance': metrics['background_temporal_variance'],
            'frame_count': frame_count
        }
        
        cap.release()
        return results
    
    def save_results(self, output_dir):
        """
        Guarda los resultados en un archivo CSV y genera gráficos comparativos.
        
        Args:
            output_dir (str): Directorio donde guardar los resultados
        """
        if not self.results:
            print("No hay resultados para guardar")
            return
        
        # Crear directorio si no existe
        Path(output_dir).mkdir(parents=True, exist_ok=True)
        
        # Convertir resultados a DataFrame
        results_df = pd.DataFrame.from_dict(self.results, orient='index')
        
        # Guardar como CSV
        csv_path = os.path.join(output_dir, 'piv_video_metrics.csv')
        results_df.to_csv(csv_path)
        print(f"Resultados guardados en {csv_path}")
        
        # Generar gráficos comparativos
        self._generate_comparison_plots(results_df, output_dir)
    
    def _generate_comparison_plots(self, df, output_dir):
        """
        Genera gráficos comparativos para las métricas principales.
        
        Args:
            df (pandas.DataFrame): DataFrame con los resultados
            output_dir (str): Directorio donde guardar los gráficos
        """
        # Métricas para graficar
        metrics_to_plot = [
            'global_contrast_std_mean', 
            'local_contrast_mean', 
            'illumination_uniformity_mean',
            'sharpness_mean', 
            'background_temporal_variance'
        ]
        
        # Crear un gráfico de barras para cada métrica
        for metric in metrics_to_plot:
            plt.figure(figsize=(12, 6))
            df[metric].plot(kind='bar', color='skyblue')
            plt.title(f'Comparación de {metric}')
            plt.ylabel('Valor')
            plt.xlabel('Video')
            plt.xticks(rotation=45, ha='right')
            plt.tight_layout()
            plt.savefig(os.path.join(output_dir, f'{metric}_comparison.png'))
            plt.close()
        
        # Crear un gráfico de radar para comparar todas las métricas normalizadas
        self._create_radar_chart(df, metrics_to_plot, output_dir)
    
    def _create_radar_chart(self, df, metrics, output_dir):
        """
        Crea un gráfico de radar para comparar todas las métricas normalizadas.
        
        Args:
            df (pandas.DataFrame): DataFrame con los resultados
            metrics (list): Lista de métricas a incluir en el radar
            output_dir (str): Directorio donde guardar el gráfico
        """
        # Normalizar las métricas
        df_norm = df.copy()
        for metric in metrics:
            if df[metric].max() > 0:
                df_norm[metric] = df[metric] / df[metric].max()
        
        # Configurar el gráfico de radar
        n_videos = len(df)
        if n_videos > 0:
            n_metrics = len(metrics)
            angles = np.linspace(0, 2*np.pi, n_metrics, endpoint=False).tolist()
            angles += angles[:1]  # Cerrar el círculo
            
            fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(polar=True))
            
            # Dibujar para cada video
            for i, video in enumerate(df.index):
                values = df_norm.loc[video, metrics].values.tolist()
                values += values[:1]  # Cerrar el círculo
                
                ax.plot(angles, values, linewidth=2, linestyle='solid', label=video)
                ax.fill(angles, values, alpha=0.1)
            
            # Configurar etiquetas y leyenda
            ax.set_theta_offset(np.pi / 2)
            ax.set_theta_direction(-1)
            ax.set_xticks(angles[:-1])
            ax.set_xticklabels([m.replace('_', ' ') for m in metrics])
            
            plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
            plt.title('Comparación de métricas normalizadas', size=15)
            plt.tight_layout()
            
            plt.savefig(os.path.join(output_dir, 'radar_comparison.png'))
            plt.close()

def create_roi_mask(frame_shape, roi_coordinates=None):
    """
    Crea una máscara de la región de interés (ROI).
    
    Args:
        frame_shape (tuple): Forma del frame (alto, ancho)
        roi_coordinates (list, optional): Lista de coordenadas (x, y) que definen el polígono ROI
                                         Si es None, se asume todo el frame como ROI.
    
    Returns:
        numpy.ndarray: Máscara binaria donde 1 representa la ROI
    """
    height, width = frame_shape
    
    if roi_coordinates is None:
        # Si no se proporcionan coordenadas, todo el frame es ROI
        return np.ones((height, width), dtype=np.uint8)
    
    # Crear máscara a partir de coordenadas de polígono
    mask = np.zeros((height, width), dtype=np.uint8)
    roi_coords = np.array(roi_coordinates, dtype=np.int32)
    cv2.fillPoly(mask, [roi_coords], 1)
    
    return mask

def main():
    """
    Función principal que ejecuta el análisis de videos PIV.
    """
    # Configuración
    videos_dir = 'videos'
    output_dir = 'metricas'
    
    # Ejemplo de coordenadas ROI (reemplazar con las reales)
    # Formato: [(x1, y1), (x2, y2), ...]
    roi_coordinates = None  # Por defecto, todo el frame es ROI
    
    # Crear un analizador de videos
    analyzer = PIVVideoAnalyzer(videos_dir)
    
    # Crear máscara ROI (opcional)
    # Para esto necesitamos obtener primero las dimensiones de un frame
    sample_video = [f for f in os.listdir(videos_dir) 
                   if f.endswith(('.mp4', '.avi', '.mov'))][0]
    cap = cv2.VideoCapture(os.path.join(videos_dir, sample_video))
    ret, frame = cap.read()
    cap.release()
    
    if ret:
        # Crear máscara con las dimensiones del frame
        if len(frame.shape) == 3:
            frame_shape = (frame.shape[0], frame.shape[1])
        else:
            frame_shape = frame.shape
            
        roi_mask = create_roi_mask(frame_shape, roi_coordinates)
        
        # Analizar videos
        results = analyzer.analyze_videos(roi_mask)
        
        # Guardar y visualizar resultados
        analyzer.save_results(output_dir)
        
        # Imprimir resultados
        print("\nResumen de resultados:")
        for video, metrics in results.items():
            print(f"\nVideo: {video}")
            for metric, value in metrics.items():
                print(f"  {metric}: {value:.4f}" if isinstance(value, float) else f"  {metric}: {value}")
    else:
        print("Error: No se pudo leer el primer frame para determinar las dimensiones")

if __name__ == "__main__":
    main()

Analizando video: FULL_pre55.mp4
Analizando video: FULL_sr_8_5.mp4
Error: Las dimensiones de la máscara no coinciden con las del video
Analizando video: FULL_CLAHE_8_5.mp4
Analizando video: FULL_post30.mp4
Resultados guardados en metricas/piv_video_metrics.csv

Resumen de resultados:

Video: FULL_pre55.mp4
  global_contrast_std_mean: 44.5868
  global_contrast_std_var: 1.0610
  global_contrast_range_mean: 254.2605
  global_contrast_range_var: 1.2578
  local_contrast_mean: 50.9174
  local_contrast_var: 1.6340
  illumination_uniformity_mean: 42.4930
  illumination_uniformity_var: 1.3093
  sharpness_mean: 514.4702
  sharpness_var: 1728.6710
  background_temporal_variance: 0
  frame_count: 261

Video: FULL_sr_8_5.mp4
  global_contrast_std_mean: 49.0482
  global_contrast_std_var: 0.6296
  global_contrast_range_mean: 254.9272
  global_contrast_range_var: 0.1824
  local_contrast_mean: 48.9699
  local_contrast_var: 0.2442
  illumination_uniformity_mean: 43.8753
  illumination_uniformity_var: 0.